In [1]:
# Dependecies
import requests
import json
import pandas as pd
import pymongo
from pymongo import MongoClient
from pprint import pprint

In [2]:
recipes = pd.read_csv("Resources/RAW_recipes.csv")
interactions = pd.read_csv("Resources/RAW_interactions.csv")

## Recipe Data

In [3]:
recipes = recipes.drop(['contributor_id', 'submitted'], axis=1)
recipes = recipes.dropna()
recipes = recipes.rename(columns = {"id": "recipe_id"})

# Drop items with zero cooking minutes
drop_times = (recipes.loc[recipes['minutes']==0]).index
recipes.drop(drop_times, inplace = True)
# Dropping Long preparation recipes
drop_recipes = (recipes.loc[(recipes['recipe_id'] == 261647) | (recipes['recipe_id'] == 447963)]).index
recipes.drop(drop_recipes, inplace = True)

In [4]:
recipes.head()

,name,recipe_id,minutes,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Interactions

In [5]:
interactions = interactions.drop(['date'], axis=1)
interactions = interactions.dropna()
combined_interactions = pd.merge(interactions, recipes[['recipe_id', 'name', 'minutes']], on = 'recipe_id')


# Drop items with zero cooking minutes
drop_times = (recipes.loc[recipes['minutes']==0]).index
recipes.drop(drop_times, inplace = True)
# Dropping Long preparation recipes
drop_recipes = (recipes.loc[(recipes['recipe_id'] == 261647) | (recipes['recipe_id'] == 447963)]).index
recipes.drop(drop_recipes, inplace = True)

In [6]:
combined_interactions.head()

,user_id,recipe_id,rating,review,name,minutes
0,38094,40893,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,495
1,1293707,40893,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,495
2,126440,85009,5,I made the Mexican topping and took it to bunk...,baked potato toppings,10
3,57222,85009,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,10
4,52282,120345,4,very very sweet. after i waited the 2 days i b...,sugared raspberries,10


## Convert to JSON

In [7]:
recipes_json = recipes.set_index('recipe_id').to_json(orient='index')
#interactions_json = combined_interactions.to_json(orient='index')

In [8]:
# Convert json string to DICT
recipes_json = eval(recipes_json)

In [9]:
type(recipes_json)

dict

## TESTING

In [19]:
test_df = recipes.head(1)
test_df

,name,recipe_id,minutes,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7


In [20]:
test_json = eval(test_df.set_index('recipe_id').to_json(orient='index'))

In [21]:
test_json

{'137739': {'name': 'arriba   baked winter squash mexican style',
  'minutes': 55,
  'tags': "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
  'nutrition': '[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]',
  'n_steps': 11,
  'steps': "['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you

In [22]:
type(test_json)

dict

## MongoDB

In [23]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.recipe_db
collection = db.recipes



In [27]:
collection.insert_many(recipes_json, ordered=False)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping